In [1]:
print('test')

test


In [ ]:
# import library
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Tentukan direktori dataset
dataset_dir = '/home/mpfaezol/Downloads/Random Code/save brain code/chrome ektension for save brain/build model/dataset'  # Ganti dengan path dataset kamu
img_size = 224  # Ukuran gambar yang diinginkan untuk model (biasanya 224x224)
    

In [ ]:
# Membaca gambar dan mengubah ukurannya
def load_images_from_folder(folder):
    images = []
    labels = []
    for label, category in enumerate(os.listdir(folder)):
        category_path = os.path.join(folder, category)
        if os.path.isdir(category_path):
            for filename in os.listdir(category_path):
                img_path = os.path.join(category_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))  # Resize gambar
                images.append(img)
                labels.append(label)  # 0 untuk non-porn, 1 untuk porn
    return np.array(images), np.array(labels)

# Load dataset
images, labels = load_images_from_folder(dataset_dir)

# Normalisasi gambar
images = images / 255.0  # Normalisasi antara 0 dan 1

In [ ]:
# Split dataset menjadi training dan validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


# Membangun Model CNN (Convolutional Neural Network)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularisasi dengan Dropout
    
    Dense(1, activation='sigmoid')  # Output 1 (porn/non-porn)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Melihat ringkasan model
model.summary()


# LATIH MODEL

In [ ]:
# Melatih model dengan data yang telah dipersiapkan
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    batch_size=32
)

# EVALUASI MODEL

In [ ]:
# Evaluasi model di data validasi
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')


# SIMPAN MODEL

In [ ]:
# Simpan model yang sudah dilatih
model.save('porn_detection_model.h5')

# PREDICTION MODEL

In [ ]:
# Muat model
model = tf.keras.models.load_model('porn_detection_model.h5')

# Menggunakan model untuk prediksi gambar baru
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_size, img_size)) / 255.0  # Normalisasi gambar
    img = np.expand_dims(img, axis=0)  # Tambahkan batch dimension
    prediction = model.predict(img)
    return prediction[0][0]

image_path = 'path/to/your/test_image.jpg'
prediction = predict_image(image_path)
print(f'Prediction: {"Porn" if prediction > 0.5 else "Non-Porn"}')